# Up and Running with Tensorflow

"In this chapter, we will go through the basics of Tensorflow, from installation to creating, running, saving, and visualizing simple computational graphs."

In [1]:
import tensorflow as tf
tf.__version__

C:\Users\Tim\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Tim\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Tim\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Tim\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type

'1.12.0'

In [3]:
x = tf.Variable(3, name='x')
y = tf.Variable(4, name='y')
f = x*x*y + y + 2
type(f)

tensorflow.python.framework.ops.Tensor

In [4]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()
    print(result)

42


Instead of manually running the initializer for every single variable, you can use the `global_variables_initializer()` function. Note that it does not actually perform the initialization immediately, but rather creates a node in the graph that will initialize all variable when it is run:

In [6]:
init = tf.global_variables_initializer() # preparing an init node
with tf.Session() as sess:
    init.run() # initialize all variables
    result = f.eval()
    print(result)

42


### Managing Graphs

Any node you create is automatically added to the default graph.

In most cases this is fine, but sometimes you may want to manage multiple independent graphs. You can do this by creating a new `Graph` and temporarily making it the default graph inside a `with` block:

In [8]:
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)

In [10]:
x2.graph is graph

True

In [11]:
x2.graph is tf.get_default_graph()

False

### Lifecycle of a Node Value

Look at this bit of code:

In [12]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3

Notice that the *y* AND *z* nodes relies on *x* and *w*.
TF will compute the value of x, then, w, then y, but then will have to recompute x and w before computing z.

"If you want to evaluate `y` and `z` efficiently, without evaluating `w` and `x` twice as in the previous code, you must ask TensorFlow to evaluate `y` and `z` in just one graph run, as shown in the following code:"

In [13]:
with tf.Session() as sess:
    y_val, z_val = sess.run([y,z])
    print(y_val, z_val)

10 15


Here, we will demonstrate how to do linear regression (simply using the Normal equation) with TF:

In [3]:
import numpy as np
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
m,n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m,1)),housing.data]

In [ ]:
x = tf.constant(housing_data_plus_bias, dtype=tf.float32, name='x')
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name='y')
xt = tf.transpose(x)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(xt,x)), xt), y)

In [16]:
with tf.Session() as sess:
    theta_value = theta.eval()
    print(theta_value)

[[-3.7185181e+01]
 [ 4.3633747e-01]
 [ 9.3952334e-03]
 [-1.0711310e-01]
 [ 6.4479220e-01]
 [-4.0338000e-06]
 [-3.7813708e-03]
 [-4.2348403e-01]
 [-4.3721911e-01]]


### Implementing Gradient Descent

"Let's try using Batch Gradient Descent instead of the Normal Equation."

#### Manually Computing the Gradients

In [4]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data_plus_bias = scaler.fit_transform(housing_data_plus_bias)

In [22]:
n_epochs = 1000
learning_rate = 0.01


x = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='x')
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform([n+1,1],-1.0,1.0), name='theta')
y_pred = tf.matmul(x, theta, name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
gradients = 2/m * tf.matmul(tf.transpose(x), error)
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

sess = tf.InteractiveSession()
init.run()
for epoch in range(n_epochs):
    if epoch % 100 == 0:
        print('Epoch', epoch, 'MSE =', mse.eval())
    sess.run(training_op)

best_theta = theta.eval()
sess.close()

C:\Users\Tim\Anaconda3\lib\site-packages\tensorflow\python\client\session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


Epoch 0 MSE = 8.940121
Epoch 100 MSE = 5.073209
Epoch 200 MSE = 4.964173
Epoch 300 MSE = 4.9186454
Epoch 400 MSE = 4.886637
Epoch 500 MSE = 4.863559
Epoch 600 MSE = 4.846886
Epoch 700 MSE = 4.834839
Epoch 800 MSE = 4.826128
Epoch 900 MSE = 4.8198304


#### Using autodiff

"The preceding code works fine, but it requires mathematically deriving the gradients from the cost function (MSE). In the case of Linear Regression, it is reasonably easy, but if you had to do this with deep neural networks you would get quite a headache... Fortunately, TensorFlow's autodiff feature comes to the rescue: it can automatically and efficiently compute the gradients for you. Simply replace the `gradients = ...` line in the Gradient Descent code in the previous section with the following line:"

`gradients = tf.gradients(mse, [theta])[0]`

In [24]:
n_epochs = 1000
learning_rate = 0.01


x = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='x')
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform([n+1,1],-1.0,1.0), name='theta')
y_pred = tf.matmul(x, theta, name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
gradients = tf.gradients(mse, [theta])[0]
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

sess = tf.InteractiveSession()
init.run()
for epoch in range(n_epochs):
    if epoch % 100 == 0:
        print('Epoch', epoch, 'MSE =', mse.eval())
    sess.run(training_op)

best_theta = theta.eval()
sess.close()

C:\Users\Tim\Anaconda3\lib\site-packages\tensorflow\python\client\session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


Epoch 0 MSE = 10.676546
Epoch 100 MSE = 4.9156957
Epoch 200 MSE = 4.845221
Epoch 300 MSE = 4.8338785
Epoch 400 MSE = 4.826566
Epoch 500 MSE = 4.821135
Epoch 600 MSE = 4.8170424
Epoch 700 MSE = 4.813942
Epoch 800 MSE = 4.811581
Epoch 900 MSE = 4.809775


#### Using an Optimizer

Along with computing the gradient for you, "[tensorflow] also provides a number of optimizers out of the box... Simply replace the preceding `gradients = ...` and `training_op = ...` with:

In [26]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(mse)

### Feeding the Data to the Training Algorithm

"Let's try to modify the previous code to implement Mini-batch Gradient Descent. For this, we need a way to replace X and y at every iteration with the next mini-batch. The simplest way to do this is to use placeholder nodes. These nodes are special because they don't actually perform any computation, they just output the data you tell them to output at runtime."

"To create a placeholder node, you must call the `placeholder()` function and specify the output tensor's data type. Optionally, you can also specify its shape, if you want to enforce it."

In [28]:
A = tf.placeholder(tf.float32, shape=(None,3))
B = A + 5

sess = tf.InteractiveSession()
print( B.eval(feed_dict={A: [[1,2,3]]}) )
print( B.eval(feed_dict={A: [[4,5,6],[7,8,9]]}) )
sess.close()

[[6. 7. 8.]]
[[ 9. 10. 11.]
 [12. 13. 14.]]


C:\Users\Tim\Anaconda3\lib\site-packages\tensorflow\python\client\session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


Now when implementing mini-batches, we need x and y to be placeholders:

In [29]:
x = tf.placeholder(tf.float32, shape=(None, n+1), name='x')
y = tf.placeholder(tf.float32, shape=(None, 1), name='y')

Computing the number of total batches we need:

In [30]:
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

"Finally, in the execution phase, fetch the mini-batches one by one, then provide the value of x and y via the `feed_dict` parameter when evaluating a node that depends on either of them."

In [ ]:
# Don't actually run this, it's just psuedo code and won't work

def fetch_batch(epoch, batch_index, batch_size):
    #[...] load the data from the disk
    return x_batch, y_batch

sess = tf.InteractiveSession()
sess.run(init)
for epoch in range(n_epochs):
    for batch_index in range(n_batches):
        x_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
        sess.run(training_op, feed_dict={x: x_batch, y: y_batch})
best_theta = theta.eval()

### Saving and Restoring Models

"TensorFlow makes saving and restoring a model very easy. Just create a `Saver` node at the end of the construction phase (after all the variable nodes are created); then, in the execution phase, just call its `save()` method whenever you want to save the model, passing it the session and path of the checkpoint file"

In [ ]:
# During construction
saver = tf.train.Saver()

# Example execution phase:
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch % 100 == 0: #checkpoint every 100 epochs
            save_path = saver.save(sess, '/tmp/my_model.ckpt')
        sess.run(training_op)
    best_theta = theta.eval()
    save_path = saver.save(sess, '/tmp/my_model_final.ckpt')

"Restoring a model is just as easy: you create a `Saver` at the end of the construction phase just like before, but then at the beginning of the execution phase, instead of initializing the variables using the `init` node, you call the `restore()` method of the `Saver` object:"

In [ ]:
with tf.Session() as sess:
    saver.restore(sess, '/tmp/my_model_final.ckpt')
    # [...]

"By default, a `Saver` saves and restores all variables under their own name, but if you need more control, you can specigy which variables to save or restore, and what names to use:"

In [31]:
saver = tf.train.Saver({'weights': theta})

"By default, the `save()` method also saves the structure of the graph in a second file with the same name plus a .meta extension." If you want to restore your graph (and not just constructing it again), you can use `import_meta_graph`.

In [ ]:
saver = tf.train.import_meta_graph('/tmp/my_model_final.ckpt.meta')

with tf.Session() as sess:
    saver.restore(sess, '/tmp/my_model_final.ckpt')
    # [...]

### Visualizing the Graph and Training Curves Using Tensorboard

Let's add a bit to the start of our code so we can log the mse into Tensorboard.

In [32]:
from datetime import datetime

now = datetime.utcnow().strftime('%Y%m%d%H%M%S')
root_logdir = 'tf_logs'
logdir = '{}/run-{}/'.format(root_logdir, now)

In [33]:
mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

"Next, you need to update the execution phase to evaluate the `mse_summary` node regularly during training (e.g., every 10 mini-batches). This will output a summary that you can then write to the events file using the `file_writer`:"

In [ ]:
# [...]
for batch_index in range(n_batches):
    x_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
    if batch_index % 10 == 0:
        summary_str = mse_summary.eval(feed_dict={x: x_batch, y: y_batch})
        step = epoch * n_batches + batch_index
        file_writer.add_summary(summary_str, step)
    sess.run(training_op, feed_dict={x: x_batch, y: y_batch})
# [...]

In [34]:
file_writer.close() # Make sure to close file writer at end of program

In [6]:
# Here's the full predictor now:
from datetime import datetime

now = datetime.utcnow().strftime('%Y%m%d%H%M%S')
root_logdir = 'tf_logs'
logdir = '{}/run-{}/'.format(root_logdir, now)



n_epochs = 1000
learning_rate = 0.01


x = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='x')
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform([n+1,1],-1.0,1.0), name='theta')
y_pred = tf.matmul(x, theta, name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
gradients = tf.gradients(mse, [theta])[0]
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)
saver = tf.train.Saver()
mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

init = tf.global_variables_initializer()

sess = tf.InteractiveSession()
init.run()
for epoch in range(n_epochs):
    if epoch % 100 == 0:
        saver.save(sess, '/logs/model.ckpt')
    if epoch % 50 == 0:
        summary_str = mse_summary.eval()
        step = epoch * n_epochs
        file_writer.add_summary(summary_str, step)
    sess.run(training_op)
saver.save(sess, '/logs/model_final.ckpt')
best_theta = theta.eval()
sess.close()

C:\Users\Tim\Anaconda3\lib\site-packages\tensorflow\python\client\session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


NameError: name 'self' is not defined